# Assignment 1

## **목표**
- 데이터에서 잘못된 값을 정제할 수 있다.
- 아래에 있는 ns_book4에 대해 출판년도의 오류 정보를 정제하기 위해 3번째 cell에 코드를 완성하세요.

In [1]:
import gdown
import pandas as pd

gdown.download('https://bit.ly/3GisL6J', 'ns_book4.csv', quiet=False)
ns_book4 = pd.read_csv('ns_book4.csv', low_memory=False)[30000:40000]
ns_book4.isna().sum()

Downloading...
From: https://bit.ly/3GisL6J
To: c:\Users\dladk\Downloads\ns_book4.csv
100%|██████████| 55.5M/55.5M [00:10<00:00, 5.16MB/s]


번호            0
도서명           0
저자            2
출판사          85
발행년도          3
ISBN          0
세트 ISBN    9248
부가기호       5944
권          8690
주제분류번호     1060
도서권수          0
대출건수          0
등록일자          0
dtype: int64

In [2]:
import re
import requests
from bs4 import BeautifulSoup

def get_book_info(row):
    title = row['도서명']
    author = row['저자']
    pub = row['출판사']
    year = row['발행년도']
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(row['ISBN']))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    try:
        if pd.isna(title):
            # 클래스 이름이 'gd_name'인 a 태그의 텍스트를 가져옵니다.
            title = soup.find('a', attrs={'class':'gd_name'}) \
                    .get_text()
    except AttributeError:
        pass

    try:
        if pd.isna(author):
            # 클래스 이름이 'info_auth'인 span 태그 아래 a 태그의 텍스트를 가져옵니다.
            authors = soup.find('span', attrs={'class':'info_auth'}) \
                          .find_all('a')
            author_list = [auth.get_text() for auth in authors]
            author = ','.join(author_list)
    except AttributeError:
        pass

    try:
        if pd.isna(pub):
            # 클래스 이름이 'info_auth'인 span 태그 아래 a 태그의 텍스트를 가져옵니다.
            pub = soup.find('span', attrs={'class':'info_pub'}) \
                      .find('a') \
                      .get_text()
    except AttributeError:
        pass

    try:
        if year == -1:
            # 클래스 이름이 'info_date'인 span 태그 아래 텍스트를 가져옵니다.
            year_str = soup.find('span', attrs={'class':'info_date'}) \
                           .get_text()
            # 정규식으로 찾은 값 중에 첫 번째 것만 사용합니다.
            year = re.findall(r'\d{4}', year_str)[0]
    except AttributeError:
        pass

    return title, author, pub, year

In [3]:
# IMPLEMENT HERE
# 안의 알고리즘 내용을 채우세요

def data_fixing(ns_book4):
    """
    잘못된 값을 수정하거나 NaN 값을 채우는 함수

    :param ns_book4: data_cleaning() 함수에서 전처리된 데이터프레임
    """
    ns_book4 = ns_book4.astype({'도서권수':'int32', '대출건수': 'int32'})
    set_isbn_na_rows = ns_book4['세트 ISBN'].isna()
    ns_book4.loc[set_isbn_na_rows, '세트 ISBN'] = ''
    ns_book5 = ns_book4.replace({'발행년도':r'.*(\d{4}).*'}, r'\1', regex=True)
    unkown_year = ns_book5['발행년도'].str.contains('\D', na=True)
    ns_book5.loc[unkown_year, '발행년도'] = '-1'
    ns_book5 = ns_book5.astype({'발행년도': 'int32'})
    dangun_yy_rows = ns_book5['발행년도'].gt(4000)
    ns_book5.loc[dangun_yy_rows, '발행년도'] = ns_book5.loc[dangun_yy_rows, '발행년도'] - 2333
    dangun_year = ns_book5['발행년도'].gt(4000)
    ns_book5.loc[dangun_year, '발행년도'] = -1
    old_books = ns_book5['발행년도'].gt(0) & ns_book5['발행년도'].lt(1900)
    ns_book5.loc[old_books, '발행년도'] = -1
    na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1)
    updated_sample = ns_book5[na_rows].apply(get_book_info, axis=1, result_type ='expand')
    updated_sample.columns = ['도서명', '저자', '출판사', '발행년도']
    ns_book5.update(updated_sample)
    ns_book6 = ns_book5.dropna(subset=['도서명', '저자', '출판사'])
    ns_book6 = ns_book6[ns_book6['발행년도'] != -1]

    return ns_book6

In [4]:
# For test
ns_book6 = data_fixing(ns_book4)
ns_book6.isna().sum()

C:\Users\dladk\AppData\Local\Temp\ipykernel_12008\73630505.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2019 2019 2019 ... 2018 2018 2018]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  ns_book5.update(updated_sample)


번호            0
도서명           0
저자            0
출판사           0
발행년도          0
ISBN          0
세트 ISBN       0
부가기호       5900
권          8635
주제분류번호     1040
도서권수          0
대출건수          0
등록일자          0
dtype: int64